## Agile Hardware Design
***
# Short Fifo Formal Verification Demo

## Kevin Laeufer
### laeufer@berkeley.edu

- Berkeley PhD student
- maintainer of the [chiseltest](https://github.com/ucb-bar/chiseltest) library

**Note**: make sure you have the [z3 SMT solver](https://github.com/Z3Prover/z3) installed to be able to execute the examples on your machine. It should be available through your favorite package manager: _brew_, _dnf_, _apt_ etc.


## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.formal._
import chiseltest.experimental.observe
import chiseltest.RawTester._

In [ ]:
// this module will be part of the next chisel release
import chisel3.experimental.IO
/** Tracks random packets for formally verifying FIFOs
  *
  *  This ensures that when some data enters the FIFO, it will always be dequeued after the correct number of elements.
  *  So essentially we are verifying data integrity. Note that this does not imply that the FIFO has no bugs
  *  since e.g., a FIFO that never allows elements to be enqueued would easily pass our assertions.
  *
  *  This module was inspired by the MagicPacketTracker used in the evaluation of the following paper:
  *  Mann, Makai, and Clark Barrett. "Partial order reduction for deep bug finding in synchronous hardware.", CAV'20.
  */
object MagicPacketTracker {
  def apply[D <: chisel3.Data](enq: ValidIO[D], deq: ValidIO[D], depth: Int, debugPrint: Boolean): Unit = {
    val tracker = Module(new MagicPacketTracker(chiselTypeOf(enq.bits), depth, debugPrint))
    tracker.enq := enq
    tracker.deq := deq
    val startTracking = IO(Input(Bool()))
    tracker.startTracking := startTracking
  }
  def apply[D <: chisel3.Data](enq: ValidIO[D], deq: ValidIO[D], depth: Int): Unit =
    apply(enq, deq, depth, debugPrint = false)
  def apply[D <: chisel3.Data](enq: DecoupledIO[D], deq: DecoupledIO[D], depth: Int, debugPrint: Boolean): Unit =
    apply(asValid(enq), asValid(deq), depth, debugPrint)
  def apply[D <: chisel3.Data](enq: DecoupledIO[D], deq: DecoupledIO[D], depth: Int): Unit =
    apply(asValid(enq), asValid(deq), depth)

  private def asValid[D <: chisel3.Data](port: DecoupledIO[D]): ValidIO[D] = {
    val validIO = Wire(ValidIO(chiselTypeOf(port.bits)))
    validIO.valid := port.fire
    validIO.bits := port.bits
    validIO
  }
}

class MagicPacketTracker[D <: chisel3.Data] private (dataTpe: D, fifoDepth: Int, debugPrint: Boolean) extends Module {
  require(fifoDepth > 0, "Fifo depth needs to be positive!")
  val enq = IO(Input(ValidIO(dataTpe)))
  val deq = IO(Input(ValidIO(dataTpe)))

  // count the number of elements in the fifo
  val elementCount = RegInit(0.U(log2Ceil(fifoDepth + 1).W))
  val nextElementCount = Mux(
    enq.fire && !deq.fire,
    elementCount + 1.U,
    Mux(!enq.fire && deq.fire, elementCount - 1.U, elementCount)
  )
  elementCount := nextElementCount

  // track a random "magic" packet through the fifo
  val startTracking = IO(Input(Bool()))
  val isActive = RegInit(false.B)
  val packetValue = Reg(chiselTypeOf(enq.bits))
  val packetCount = Reg(chiselTypeOf(elementCount))

  when(!isActive && enq.fire && startTracking) {
    when(deq.fire && elementCount === 0.U) {
      assert(
        enq.bits.asUInt === deq.bits.asUInt,
        "element should pass through the fifo, but %x != %x",
        enq.bits.asUInt,
        deq.bits.asUInt
      )
    }.otherwise {
      isActive := true.B
      packetValue := enq.bits
      packetCount := nextElementCount
    }
  }

  when(isActive && deq.fire) {
    packetCount := packetCount - 1.U
    when(packetCount === 1.U) {
      assert(
        packetValue.asUInt === deq.bits.asUInt,
        "element should be dequeued in this cycle, but %x != %x",
        packetValue.asUInt,
        deq.bits.asUInt
      )
      isActive := false.B
    }
  }

  // detect element count overflow
  when(elementCount === fifoDepth.U) {
    val shouldIncrement = enq.fire && !deq.fire
    assert(
      !shouldIncrement,
      "MagicPacketTracker: element counter is overflowing %d -> %d\n" +
        "This could indicate either a bug in your FIFO design, or an insufficient depth provided to the MagicPacketTracker constructor.",
      elementCount,
      nextElementCount
    )
  }

  // printout to help debug counter examples
  if (debugPrint) {
    val cycle = RegInit(0.U(8.W)); cycle := cycle + 1.U
    printf(p"step: ${cycle} ----------------------------------\n")
    printf(p"element count: ${elementCount} -> ${nextElementCount}\n")
    when(enq.fire) { printf(p"[${enq.bits}]") }
    when(enq.fire || deq.fire) { printf(" --> ") }
    when(deq.fire) { printf(p"[${deq.bits}]") }
    when(enq.fire || deq.fire) { printf("\n") }
  }
}

# Simple Fifo

In [ ]:
class FifoIO(bitWidth: Int) extends Bundle {
  val enq = Flipped(Decoupled(UInt(bitWidth.W)))
  val deq = Decoupled(UInt(bitWidth.W))
}

In [ ]:

class SimpleFifo(val numEntries: Int, bitWidth: Int) extends Module {
  val io = IO(new FifoIO(bitWidth))
  require(numEntries > 0)
  // enqueue into lowest empty and dequeue from index 0 (head)
  val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
  val fullBits = RegInit(VecInit(Seq.fill(numEntries)(false.B)))
  val emptyBits = fullBits.map(!_)
  io.enq.ready := emptyBits.reduce( _ || _ ) // any empties?
  io.deq.valid := fullBits.head
  io.deq.bits := entries.head
  when (io.deq.fire) { // dequeue & shift up
    for (i <- 0 until numEntries - 1) {
      entries(i) := entries(i+1)
      fullBits(i) := fullBits(i+1)
    }
    fullBits.last := false.B
  }
  val writeIndex = PriorityEncoder(emptyBits)
  val writeIndex2 = Mux(io.deq.fire, writeIndex - 1.U, writeIndex)
  when (io.enq.fire) { // priority enqueue
    entries(writeIndex2) := io.enq.bits
    fullBits(writeIndex2) := true.B
  }
}

# Writing a Simple Unittest

In [ ]:
def push(clock: Clock, io: FifoIO, value: Int): Unit = {
    io.deq.ready.poke(false) ; io.enq.valid.poke(true)
    io.enq.ready.expect(true) ; io.enq.bits.poke(value)
    clock.step() 
}

def pop(clock: Clock, io: FifoIO, value: Int): Unit = {
    io.enq.valid.poke(false) ; io.deq.ready.poke(true)
    io.deq.valid.expect(true) ; io.deq.bits.expect(value)
    clock.step()
}


def runTest(clock: Clock, io: FifoIO): Unit = {
    push(clock, io, 0x123)
    push(clock, io, 0x456)
    pop(clock, io, 0x123)
    pop(clock, io, 0x456)
    clock.step()
}

test(new SimpleFifo(3, 32)) { dut =>
    runTest(dut.clock, dut.io)
}

# Visualize Behavior

In [ ]:
class FifoWrapper(makeFifo: => SimpleFifo) extends Module {
    val dut = Module(makeFifo)
    val io = IO(chiselTypeOf(dut.io)) ; io <> dut.io
    val entries = dut.entries.map(observe(_))
    val full = dut.fullBits.map(observe(_))
    val hexLen = (dut.io.enq.bits.getWidth / 4)
    val writeIndex = observe(dut.writeIndex)
    when(io.enq.fire) { printf("[%x] -> ", io.enq.bits) }
    .otherwise { printf(" " * (hexLen + 6)) }

    entries.zip(full).zipWithIndex.reverse.foreach { case ((entry, full), ii) =>
        when(full) { printf(s"$ii: [%x] ", entry) }
        .otherwise { printf(s"$ii: [${"X" * hexLen}] ") }
        when(ii.U === writeIndex && io.enq.fire) { printf("W ") }
        .otherwise { printf("  ") }
    }

    when(io.deq.fire) { printf("-> [%x]", io.deq.bits) }
    .otherwise { printf(" " * (hexLen + 5)) }

    printf("\n")
}

test(new FifoWrapper(new SimpleFifo(3, 32))) { dut =>
    runTest(dut.clock, dut.io)
}

# Formal Fifo Check

In [ ]:
class FormalFifoChecker(makeFifo: => SimpleFifo) extends FifoWrapper(makeFifo) {
    MagicPacketTracker(io.enq, io.deq, depth = dut.numEntries)
}

verify(new FormalFifoChecker(new SimpleFifo(4, 32)), Seq(BoundedCheck(8)))